In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError, Poisson, BinaryCrossentropy, SparseCategoricalCrossentropy, MeanSquaredLogarithmicError
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import Accuracy, CategoricalAccuracy
from tensorflow.keras.models import Sequential
from itertools import chain
import matplotlib.pyplot as plt
from tensorflow.python.ops import math_ops
import random
from tensorflow.keras.backend import mean
import time
from numpy import unravel_index
from copy import deepcopy
from IPython.display import clear_output

In [ ]:
class Game:
    def __init__(self, train=True, model_path=None, num_boards=1000, training_rounds=50, ships=[3], axis=0):
        self.num_boards = num_boards
        self.ships = ships
        self.training_boards = []
        self.training_rounds = training_rounds
        self.tracker = []
        self.average_game_lengths = []
        self.axis = axis
        self.compile_model = True
        if model_path != None:
            self.model = load_model(model_path)
            self.compile_model = False
        if train:
            if self.compile_model:
                self.model_init()
            for n in range(training_rounds):
                self.fill_boards()
                round = 0
                while len(self.training_boards) != 0 and round < 100:
                    round += 1
                    print(f"training cycle: {n}, round: {round}, number of active games: {len(self.training_boards)}")
                    self.run_round()
                clear_output(wait=True)
                
    def fill_boards(self):
        self.training_boards = []
        for board in range(self.num_boards):
            self.training_boards.append(self.fill_board())
            
    def model_init(self):
        if self.compile_model:
            model = Sequential([
                    keras.Input(shape=(10,10,2,1)),
                    layers.Conv3D(32, kernel_size=(9,9,1), padding="same", activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'),
                    layers.Conv3D(64, kernel_size=(7,7,1), padding="same", activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'),   
                    layers.Conv3D(128, kernel_size=(5,5,1), padding="same", activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'),   
                    layers.Conv3D(256, kernel_size=(5,5,1), padding="same", activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'),   
                    layers.Flatten(),
                    layers.Dense(256, activation='relu', kernel_initializer='random_normal', bias_initializer='zeros'),
                    layers.Dense(128, activation='relu', kernel_initializer='random_normal', bias_initializer='zeros'),
                    layers.Dense(128, activation='relu', kernel_initializer='random_normal', bias_initializer='zeros'),
                    layers.Dense(100, activation="softmax", kernel_initializer='random_normal', bias_initializer='zeros'),
                ])
            model.compile(loss=MeanSquaredLogarithmicError(),
                          optimizer=Adam(),
                          metrics=[CategoricalAccuracy()])
            self.model = model
    
        
    
    def fill_board(self):
        board = [[(0,0,0,0) for i in range(10)] for j in range(10)] 
        player = 1
        for ship in self.ships:
            ship_origin = random.choice(["left", "right"])
            orientation = random.choice(["horizontal", "vertical"])
            self.tracker.append((orientation, ship_origin))
            board = fill_ship(board, ship, player, orientation, ship_origin=ship_origin) 
        player = 2
        for ship in self.ships:
            ship_origin = random.choice(["left", "right"])
            orientation = random.choice(["horizontal", "vertical"])
            self.tracker.append((orientation, ship_origin))
            board = fill_ship(board, ship, player, orientation, ship_origin=ship_origin)
        return board
            
    def run_predictions(self):
        #player 1
        boards_player_1 = [sanitize_board_for_player(board, 1) for board in self.training_boards]
        player_1_predictions = self.model.predict(np.expand_dims(np.array(boards_player_1), axis=self.axis))
        player_1_predictions = [player_1_predictions[n] for n in range(len(player_1_predictions))]
        
        #player 2
        boards_player_2 = [sanitize_board_for_player(board, 2) for board in self.training_boards]
        player_2_predictions = self.model.predict(np.expand_dims(np.array(boards_player_2), axis=self.axis))
        player_2_predictions = [player_2_predictions[n] for n in range(len(player_2_predictions))]
        self.predictions = {
            "player_1": player_1_predictions,
            "player_2": player_2_predictions
        }
            
    def train_model(self):
        history = self.model.fit(self.training_data["player_1"]["data"],
                         self.training_data["player_1"]["labels"],
                         epochs=10,
                         batch_size = 128,
                         verbose = 1
                        )
        history = self.model.fit(self.training_data["player_2"]["data"],
                         self.training_data["player_2"]["labels"],
                         epochs=10,
                         batch_size = 128,
                         verbose = 1
                        )
    
    def prune_games(self):
        delete_indexes = []
        for n in range(len(self.training_boards)):
            if get_winner(self.training_boards[n]) != 0:
                delete_indexes.append(n)
        updated_boards = []
        for n in range(len(self.training_boards)):
            if n not in delete_indexes:
                updated_boards.append(self.training_boards[n])
        self.training_boards = updated_boards
    
    def create_training_data(self):
        #trzeba stworzyć tę samą planszę z perspektywy obu graczy.
        #dla każdego gracza usuwamy informację z planszy o których nie wie.
        #player 1
        training_data_player_1 = {
            "data": [],
            "labels": []
        }
        for board in self.training_boards:
            training_data_player_1["data"].append(sanitize_board_for_player(board, 1))
            #zaznaczamy tylko statki gracza 2
            training_data_player_1["labels"].append([board[i][j][2] for i in range(10) for j in range(10)])
        training_data_player_1["data"] = np.expand_dims(np.array(training_data_player_1["data"])/3.0, axis=self.axis)
        training_data_player_1["labels"] = np.array(training_data_player_1["labels"])/1.0
        #player 2
        training_data_player_2 = {
            "data": [],
            "labels": []
        }
        for board in self.training_boards:
            training_data_player_2["data"].append(sanitize_board_for_player(board, 2))
            #zaznaczamy tylko statki gracza 1
            training_data_player_2["labels"].append([board[i][j][0] for i in range(10) for j in range(10)])
        training_data_player_2["data"] = np.expand_dims(np.array(training_data_player_2["data"])/3.0, axis=self.axis)
        training_data_player_2["labels"] = np.array(training_data_player_2["labels"])/1.0
        self.training_data = {
                "player_1": training_data_player_1,
                "player_2": training_data_player_2
            }
        
    
    def apply_predictions(self):
        for n in range(len(self.training_boards)):
            #player 1:
            for j in range(100):
                x_p, y_p = get_2d_indecies(j, 10)
                if self.training_boards[n][x_p][y_p][1] == 1:
                    self.predictions["player_1"][n][j] = 0
            x, y = get_2d_indecies(self.predictions["player_1"][n].argmax(), 10)
            self.training_boards[n][x][y] = (self.training_boards[n][x][y][0], 1, self.training_boards[n][x][y][2],self.training_boards[n][x][y][3])
            #player 2:
            for j in range(100):
                x_p, y_p = get_2d_indecies(j, 10)
                if self.training_boards[n][x_p][y_p][3] == 1:
                    self.predictions["player_2"][n][j] = 0
            x, y = get_2d_indecies(self.predictions["player_2"][n].argmax(), 10)
            self.training_boards[n][x][y] = (self.training_boards[n][x][y][0], self.training_boards[n][x][y][1], self.training_boards[n][x][y][2],1)
        
    def calcualte_average_moves(self):
        self.training_boards = []
        iterations = 1000
        for board in range(iterations):
            self.training_boards.append(self.fill_board())
        games_in_round = []
        round = 0
        while len(self.training_boards) != 0 and round < 100:
            games_in_round.append(len(self.training_boards))
            self.run_predictions()
            self.apply_predictions()
            self.prune_games()
            round += 1
        self.average_game_lengths.append(sum(games_in_round)/iterations)
        print(f"The average game length is: {sum(games_in_round)/iterations} rounds")
        
    def run_round(self):
        # 1 uzyskać predykcje obecnego modelu
        self.run_predictions()
        # 2 apply predicitons
        self.apply_predictions()
        # 3 stworzyć training set
        self.create_training_data() 
        # 4 przeprowadzić trening modelu
        self.train_model()
        # 5 sprawdzić które gry zostały zakończone i je wyrzućić z trwających gier
        self.prune_games()

    
        

In [ ]:
# oznaczcenia punktu
# 0 - statek gracza 1
# 1 - pole trafione przez gracza 1
# 2 - statek gracza 2
# 3 - pole trafione przez gracza 2 
def sanitize_board_for_player(board, player):
    new_board = [[(0,0) for i in range(10)] for j in range(10)] 
    if player == 1:
        for i in range(10):
            for j in range(10):
                #tu jest statek gracza 2 który nie został jeszcze trafiony
                if board[i][j][2] == 1 and board[i][j][1] == 0:
                    #1 nasze strzały
                    #2 trafione statki
                    new_board[i][j] = (board[i][j][1],0)
                else:
                    new_board[i][j] = (board[i][j][1],board[i][j][2])
    if player == 2:
        for i in range(10):
            for j in range(10):
                #tu jest statek gracza 1 który nie został jeszcze trafiony
                if board[i][j][0] == 1 and board[i][j][3] == 0:
                    new_board[i][j] = (board[i][j][3],0)
                else:
                    new_board[i][j] = (board[i][j][3],board[i][j][0])
    return new_board

def fill_ship(board, ship, player, orientation, recursion_level=0, ship_origin=None):
    if ship_origin == None:
        ship_origin = random.choice(["left", "right"])
    if orientation == "vertical":
        if ship_origin == "right":
            vertical_combinations = get_vertical_combinations_right(ship)
        if ship_origin == "left":
            vertical_combinations = get_vertical_combinations_left(ship)
        random.shuffle(vertical_combinations)
        found_suitable = False
        while not found_suitable:
            is_any_of_point_wrong = False
            if len(vertical_combinations) == 0:
                new_recursion_level = recursion_level + 1
                if new_recursion_level > 3:
                    return fill_ship(board, ship, player, "horizontal", recursion_level=new_recursion_level)
                else:
                    print_board(board, 0)
                    print_board(board, 2)
                    raise ValueError("ValueError exception thrown")
            row, column = vertical_combinations.pop()
            if ship_origin == "right":
                for point in range(ship):
                    if is_player_ship(player, board[row][column + point]):
                        is_any_of_point_wrong = True
                        break
            if ship_origin == "left":
                for point in range(ship):
                    if is_player_ship(player, board[row][column - point]):
                        is_any_of_point_wrong = True
                        break
            if not is_any_of_point_wrong:
                found_suitable = True
        for point in range(ship):
            if ship_origin == "right":
                board[row][column + point] = get_player_point(player, board[row][column + point])
            if ship_origin == "left":
                board[row][column - point] = get_player_point(player, board[row][column - point])
    if orientation == "horizontal":
        if ship_origin == "right":
            horizontal_combinations = get_horizontal_combinations_right(ship)
        if ship_origin == "left":
            horizontal_combinations = get_horizontal_combinations_left(ship)
        random.shuffle(horizontal_combinations)
        found_suitable = False
        while not found_suitable:
            is_any_of_point_wrong = False
            if len(horizontal_combinations) == 0:
                new_recursion_level = recursion_level + 1
                if new_recursion_level  > 3:
                    return fill_ship(board, ship, player, "vertical", recursion_level=new_recursion_level)
                else:
                    print_board(board, 0)
                    print_board(board, 2)
                    raise ValueError("ValueError exception thrown")
            row, column = horizontal_combinations.pop()
            for point in range(ship):
                if ship_origin == "right":
                    if is_player_ship(player, board[row + point][column]):
                        is_any_of_point_wrong = True
                        break
                if ship_origin == "left":
                    if is_player_ship(player, board[row - point][column]):
                        is_any_of_point_wrong = True
                        break
            if not is_any_of_point_wrong:
                found_suitable = True
        for point in range(ship):
            if ship_origin == "right":
                board[row + point][column] = get_player_point(player, board[row + point][column])
            if ship_origin == "left":
                board[row - point][column] = get_player_point(player, board[row - point][column])
            
    return board

def is_player_ship(player, point):
    if player == 1:
        if point == (1,0,0,0):
            return True
        else:
            return False
    if player == 2:
        if point == (1,0,1,0) or point == (0,0,1,0):
            return True
        else:
            return False
def get_player_point(player, point):
    if player == 1:
        return (1, point[1], point[2], point[3])
    if player == 2:
        return (point[0], point[1], 1, point[3])

def get_vertical_combinations_right(ship):
    combinations = []
    for x in range(0, 10):
        for y in range(0, 11 - ship):
            combinations.append((x,y))
    return combinations

def get_vertical_combinations_left(ship):
    combinations = []
    for x in range(0, 10):
        for y in range(ship - 1, 10):
            combinations.append((x,y))
    return combinations

def get_horizontal_combinations_right(ship):
    combinations = []
    for x in range(0, 11 - ship):
        for y in range(0, 10):
            combinations.append((x,y))
    return combinations

def get_horizontal_combinations_left(ship):
    combinations = []
    for x in range(ship - 1, 10):
        for y in range(0, 10):
            combinations.append((x,y))
    return combinations
  
def print_board(board, index):
        action = {
            "0": "statki",
            "1": "strzały",
            "2": "statki",
            "3": "strzały"
        }
        
        player = {
            "0": "1",
            "1": "1",
            "2": "2",
            "3": "2"
        }
        printable_board = [[board[i][j][index] for i in range(10)] for j in range(10)]
        print(f"player{player[str(index)]} {action[str(index)]}")
        for row in printable_board:
            print('   '.join([str(row[i]) for i in range(len(row))]) + "\n")
            
def get_winner(board):
    #player 1
    does_player_1_have_a_valid_ship = False
    for row in board:
        for point in row:
            if point[0] == 1 and point[3] != 1:
                does_player_1_have_a_valid_ship = True
                break
    #plaer 2
    does_player_2_have_a_valid_ship = False
    for row in board:
        for point in row:
            if point[2] == 1 and point[1] != 1:
                does_player_2_have_a_valid_ship = True
                break
    #decide winner
    if does_player_2_have_a_valid_ship == False:
        return 1
    elif does_player_1_have_a_valid_ship == False:
        return 2
    else:
        return 0
    
def create_winning_board(player):
    if player == 1:
        # 1 - player 1 has valid unshot ship
        # 2 - player 1 has shot ship
        # 3 - player 2 has shot ship
        # valid player 2 ship is ommitted
        board = [[random.choice([(1,0,0,0), (1,0,0,1), (0,1,1,0)]) for i in range(10)] for j in range(10)] 
    if player == 2:
        # 1 - player 2 has valid unshot ship
        # 2 - player 2 has shot ship
        # 3 - player 1 has shot ship
        # valid player 2 ship is ommitted
        board = [[random.choice([(0,0,1,0), (0,1,1,0), (1,0,0,1)]) for i in range(10)] for j in range(10)] 
    return board

def get_2d_indecies(index, side_length):
    if index == 0:
        return ((0,0))
    if index % side_length == 0:
        return((index//side_length , 0))
    else:
        return((index//side_length, index - (index // side_length) * side_length))
def show_ship_distribution(ships, iterations):
    scores = [[0 for i in range(10)] for j in range(10)] 
    for n in range(iterations):
        board = [[(0,0,0,0) for i in range(10)] for j in range(10)] 
        player = 1
        for ship in ships:
            orientation = random.choice(["horizontal", "vertical"])
            board = fill_ship(board, ship, player, orientation) 
        player = 2
        for ship in ships:
            orientation = random.choice(["horizontal", "vertical"])
            board = fill_ship(board, ship, player, orientation)
        for i in range(10):
            for j in range(10):
                scores[i][j] += board[i][j][0]
                scores[i][j] += board[i][j][2]
    printable_scores = [[0 for i in range(10)] for j in range(10)] 
    for i in range(10):
        for j in range(10):
            printable_scores[i][j] = scores[i][j]/(iterations * 2 * sum(ships)/100)
    for row in printable_scores:
        print('   '.join(["%.2f" % row[i] for i in range(len(row))]) + "\n")

In [ ]:
game = Game(num_boards=256, training_rounds=10, ships=[5,4,3,2], axis=4, model_path="../input/model-pretrained/model_10x10_2345_2 (1).h5") 
print("end")

In [ ]:
game.calcualte_average_moves()

In [ ]:
game.model.summary()

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

vegetables = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
farmers = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

#harvest = game.model.predict( np.expand_dims(np.array([sanitize_board_for_player(game.training_boards[3],1)]), 4))
harvest = np.array([game.training_boards[3][i][j][2] for i in range(10) for j in range(10)])
print(harvest.shape)
harvest = np.reshape(harvest, (10,10))

fig, ax = plt.subplots()
im = ax.imshow(harvest)

# We want to show all ticks...
ax.set_xticks(np.arange(len(farmers)))
ax.set_yticks(np.arange(len(vegetables)))
# ... and label them with the respective list entries
ax.set_xticklabels(farmers)
ax.set_yticklabels(vegetables)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

ax.set_title("actual ship distribution")
fig.tight_layout()
plt.show()